In [1]:
# Global import

import datetime as dt
import pytz

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [2]:
# Global variables

S3_DATASET = "s3://noos-prod-neptune-services/Store/Datasets/"

ASSET_NAME = "PWRTE"
PORTFOLIO_NAME = "TEST_FR"
PUBLISHED_AT = dt.datetime(2021, 6, 1)

TABLE_COLUMNS = ["calendar_delivery", "intraday_delivery", "quantity"]

# PAGE 1

In [3]:
# Retrieve agreements

agreement_volumes_df = pd.read_parquet(S3_DATASET + "NOOS/consommation_tot_france_random_1k.parquet")

In [4]:
# Retrieve hedges

QUERY_PARAMS = {
    "asset": ASSET_NAME,
    "portfolio_name": PORTFOLIO_NAME,
}

hedges_df = nptc.filter_forward_contracts(**QUERY_PARAMS)

In [9]:
# Retrieve products

QUERY_PARAMS = {
    "asset": ASSET_NAME,
    "published_at": PUBLISHED_AT,
    # At least day products
    "min_delivery_period": dt.timedelta(hours=8),
}

products_df = nptc.retrieve_forward_fixings(**QUERY_PARAMS)

## SECTION 1

### CARD 1

In [10]:
# Extract hedge instruments

indices_df = npta.domain.products.collections.IndexSet.from_pandas(products_df).to_pandas()

base_deliveries_ts = indices_df[indices_df.intraday_delivery == "BASE"].calendar_delivery.reset_index(drop=True)
peak_deliveries_ts = indices_df[indices_df.intraday_delivery == "PEAK"].calendar_delivery.reset_index(drop=True)

In [11]:
# Extract initial hedge volumes

hedge_volumes_df = npta.get_volumes_and_costs_from_trades(hedges_df)

In [12]:
# Shared scatter configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Local delivery',
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='MWh',
        ),
        fixedrange=True,
    ),
    template="noos_watermark+noos_pastel+hoover_xunified+min_modebar",
)

In [13]:
all_agreements = go.Scatter(
    x = agreement_volumes_df.index,
    y = agreement_volumes_df.consumption,
    line_color=themes.NOOS_HTML_COLORS["primary"],
    name = "Agreement volumes",
    visible = True
)

initial_trades = go.Scatter(
    x = hedge_volumes_df.index,
    y = hedge_volumes_df.total_volume,
    line_color=themes.NOOS_HTML_COLORS["accent"],
    name = "Hedge volumes",
    fill='tozeroy',
    visible = True
)

trades_with_hedges = go.Scatter(
    x = hedge_volumes_df.index,
    y = [],
    line_color=themes.NOOS_HTML_COLORS["error"],
    name = "Suggested hedge volumes",
    mode='lines+markers',
    visible = False
)


fig = go.FigureWidget(
    data=[
        all_agreements,
        initial_trades,
        trades_with_hedges,
    ],
    layout=GO_LAYOUT,
)

# DASH

In [14]:
import jupyter_dash

from dash import dcc, html

import dash_mdc_neptune as mdc

## LAYOUT

In [15]:
# Display graph & summary in sections
graph = dcc.Graph(
    id="graph",
    figure=fig,
    style={"height": "100%", "width": "100%"},
)

table = mdc.Table(
    id="table",
    columns=[{"field": c.replace("_", " ").capitalize(), "width": 200} for c in TABLE_COLUMNS],
    rows=[],
)

In [16]:
# Dashboard sections

section_summary = mdc.Section(
    id="section-summary",
    children=table,
    size=3,
    cards=[{"title": "Shaping summary"}],
)

section_graph = mdc.Section(
    id="section-graph",
    children=graph,
    cards=[{"title": "Portfolio profile"}],
)

In [17]:
# Dashboard sidebar

base_dropdown = mdc.Dropdown(
    id="dropdown-base",
    labelText="Base",
    width=180,
    options=base_deliveries_ts.to_list(),
)
peak_dropdown = mdc.Dropdown(
    id="dropdown-peak",
    labelText="Peak",
    width=180,
    options=peak_deliveries_ts.to_list(),
)

strategy_toggle = mdc.Toggle(
    id="toggle-strategy",
    options=["Volume", "Value"],
)

sidebar = mdc.SideBar(
    children=[base_dropdown, peak_dropdown, strategy_toggle],
    settings=["Select product", "", "Select strategy"],
)

In [18]:
# Dashboard layout

page = mdc.Page(
    orientation="columns",
    children=[section_summary, section_graph],
)
navbar = mdc.NavBar(title="Portfolio Position Optimization")
dashboard = mdc.Dashboard(children=[navbar, sidebar, page])

store = dcc.Store(id='session-store', storage_type='session')

layout = mdc.Box(children=[store, dashboard])

## APP

In [23]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate


app = JupyterDash(__name__)
app.layout = layout

In [24]:
# Store hedges and volumes in browser


def get_initial_hedges(base_products, peak_products, asset_name=ASSET_NAME):
    df = pd.DataFrame()
    calendar_products = []
    intraday_products = []

    if base_products:
        calendar_products += base_deliveries_ts.loc[base_products].to_list()
        intraday_products += ["BASE"] * len(base_products)
    if peak_products:
        calendar_products += peak_deliveries_ts.loc[peak_products].to_list()
        intraday_products += ["PEAK"] * len(peak_products)

    df["calendar_delivery"] = calendar_products
    df["intraday_delivery"] = intraday_products
    df["asset"] = asset_name
    df["quantity"] = 1

    return df


def get_suggested_hedges(base_products, peak_products, net_volumes):
    initial_hedges_df = get_initial_hedges(base_products, peak_products)
    initial_hedge_volumes_df = npta.get_volumes_from_hedges(initial_hedges_df)

    df = npta.get_hedges_from_volumes(
        net_volumes.reindex(initial_hedge_volumes_df.index).fillna(0),
        products=initial_hedges_df,
    )
    df = df.astype({'quantity': 'int'})
    df = df[~(df.quantity == 0)]

    return df


def get_default_store_state():
    net_volumes =  agreement_volumes_df.consumption - hedge_volumes_df.total_volume
    net_volumes = net_volumes.fillna(0)

    return {
        "hedges": '{}',
        "hedge_volumes": '[]',
        "net_volumes": net_volumes.to_json(orient="index", date_format="iso"),
    }


def update_store_state(store, base_products, peak_products):
    # Fetch hedge instruments
    net_volumes = pd.read_json(store["net_volumes"], typ='series')

    # Apply hedging strategy
    suggested_hedges_df = get_suggested_hedges(base_products, peak_products, net_volumes)
    suggested_hedge_volumes_df = npta.get_volumes_from_hedges(suggested_hedges_df)

    store["hedges"] = suggested_hedges_df[TABLE_COLUMNS].to_json(orient="records")
    store["hedge_volumes"] = suggested_hedge_volumes_df.to_json(orient="index", date_format="iso")
    return store


@app.callback(
    Output(component_id='session-store', component_property='data'),
    [
        Input(component_id='dropdown-base', component_property='selected'),
        Input(component_id='dropdown-peak', component_property='selected'),
    ],
    State(component_id='session-store', component_property='data'),
)
def on_select(base, peak, store):
    # Initial callback on page load
    if not store:
        return get_default_store_state()

    # No selection
    data = store
    if not base and not peak:
        raise PreventUpdate
    
    # Update store state
    return update_store_state(data, base, peak)

In [25]:
# Update hedges and volumes in components


@app.callback(
    [
        Output(component_id='table', component_property='rows'),
        Output(component_id='graph', component_property='figure'),
    ],
    Input(component_id='session-store', component_property='modified_timestamp'),
    State(component_id='session-store', component_property='data'),
)
def update_components(timestamp, store):
    # Initial callback on page load
    if timestamp is None or store is None:
        raise PreventUpdate

    # Update table
    suggested_hedges_df = pd.read_json(store["hedges"], typ='frame')
    rows = suggested_hedges_df.to_dict(orient="records")

    # Update graph
    suggested_hedge_volumes_df = pd.read_json(store["hedge_volumes"], typ='series')
    fig.data[2].visible = True
    fig.data[2].y = hedge_volumes_df.total_volume + suggested_hedge_volumes_df

    return [rows, fig]

In [27]:
app.run_server(
    mode='external', host="0.0.0.0", port=8001, debug=False
)

Dash app running on http://0.0.0.0:8001/


/opt/conda/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

